In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point,Polygon
import csv
import random
from geopandas.tools import sjoin
import multiprocessing
import time
from pathos.multiprocessing import ProcessingPool as Pool
from prettytable import PrettyTable

In [2]:
!python --version
import platform
print(platform.platform())
print("cpu cores: {0}".format(multiprocessing.cpu_count()))

Python 3.7.3
Darwin-18.6.0-x86_64-i386-64bit
cpu cores: 4


In [3]:
lat = []
lon = []
data_points = 100000
for i in range(data_points):
    x = random.uniform(-90,90)
    lat.append(x)
    
    
for i in range(data_points):
    x = random.uniform(-180,180)
    lon.append(x)

df_cord = pd.DataFrame(lon , columns = ['Lon'])
df_cord['Lat'] = lat
geometry = [Point(xy) for xy in zip(df_cord.Lon, df_cord.Lat)]
df_cord = df_cord.drop(['Lon' , 'Lat' ], axis=1)
crs = {'init': 'epsg:4326'}
gdf = geopandas.GeoDataFrame(df_cord, crs=crs, geometry=geometry)

In [4]:
print(gdf.shape)

(100000, 1)


In [5]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# ax = world.plot( color='white', edgecolor='black' ,figsize = (40,30))
# gdf.plot(ax=ax, color='red')
# plt.title("World map showing location of given coordinate")
# print(world.head(3))
# plt.show()

In [6]:
df_new = world[['continent' , 'name' , 'geometry']]

In [7]:
def computation(data_split):
    for index,row in data_split.iterrows():
        filter = df_new['geometry'].contains(row['geometry'])
        x = df_new.loc[filter]
        value_c = x['name'].to_string(index = False)
        data_split.loc[index, 'Country'] = value_c if value_c != 'Series([], )' else np.NaN
    return data_split

In [8]:
#computation(gdf)

In [9]:
start_time = time.time()  

cores = 32
partitions = 32


data_split = np.array_split(gdf, cores, axis=0)
pool = Pool(cores)
for i in range(len(data_split)):
    print(data_split[i].shape)
df_out = np.vstack(pool.map(computation, data_split))
pool.close()
pool.join()
pool.clear()
print("--- %s seconds ---" % (time.time() - start_time))

(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
(3125, 1)
--- 183.68983602523804 seconds ---


In [10]:
op  = pd.DataFrame(data = df_out)
print(op)

                                                    0                 1
0         POINT (164.2954037092926 85.89183015170727)               NaN
1          POINT (136.355165572281 72.89629359293937)               NaN
2        POINT (-98.26515234379488 54.49722717074582)            Canada
3        POINT (-139.225293784404 -33.34597115111665)               NaN
4        POINT (54.96836321304835 -33.82780483374481)               NaN
5       POINT (-97.08654113291566 -30.40771487099786)               NaN
6         POINT (85.82116392749214 27.93633091746344)             Nepal
7        POINT (-130.2923104363879 74.96457221016703)               NaN
8      POINT (-0.5194762779173061 -61.34264935703615)               NaN
9         POINT (29.21840626428059 12.48343851891812)             Sudan
10     POINT (-0.8791770623273578 -69.84084543208823)               NaN
11      POINT (-48.82479096033819 -51.83831674273843)               NaN
12        POINT (118.5215908512287 88.48466654772125)           

In [13]:
x = PrettyTable()

x.field_names = ["S.No." ," Number of Partitions", "Data Points per partition", "Time taken to run(minutes)"]
x.add_row([0, 0, 1000000, 155])
x.add_row([1, 4, 250000, 55])
x.add_row([2, 8, 125000, 40])
x.add_row([3, 12, 83333, 36])                 
x.add_row([4, 16, 6250, 35.5])
x.add_row([5, 32, 3125, 30.5])

In [14]:
print("For 1 million data points")
print(x)

For 1 million data points
+-------+-----------------------+---------------------------+----------------------------+
| S.No. |  Number of Partitions | Data Points per partition | Time taken to run(minutes) |
+-------+-----------------------+---------------------------+----------------------------+
|   0   |           0           |          1000000          |            155             |
|   1   |           4           |           250000          |             55             |
|   2   |           8           |           125000          |             40             |
|   3   |           12          |           83333           |             36             |
|   4   |           16          |            6250           |            35.5            |
|   5   |           32          |            3125           |            30.5            |
+-------+-----------------------+---------------------------+----------------------------+
